In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO"

## 1. GSEA analysis for each set of comparison

In [8]:
##########----- Parameters-----##########
analysis_name <- c("0", "all")
genome <- "mm"
#########################################
analysis_subdir <- paste(paste(analysis_name[1], analysis_name[2], sep="-"), "Scanpy_out", sep="_")
sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "4_DE_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)

###----- Find all de files
de.out.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "2_DE")
de.files <- list.files(de.out.dir, pattern="*differential.csv", full.names=TRUE,recursive=TRUE)

use.cols <- c("gene_names", "t-test_overestim_var_logfc", "t-test_overestim_var_padj")

###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files

[1] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/2020_mouse_CD8_mm_sigs.csv"    
[2] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD4_mm_sigs.csv"               
[3] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD4-ifn_mm_sigs.csv"           
[4] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD8_mm_sigs.csv"               
[5] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Exp391_CD8_mm_sigs.csv"        
[6] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Wolski_curated_CD8_mm_sigs.csv"

In [4]:
###----- Il2 dependent analysis
#gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv')

In [9]:
completed.df <- data.frame()
for (gs.file.i in c("file", gs.files)) {
    gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
    completed.df[gs.file.base] <- character(0)
}
completed.name <- file.path(sum.dir, "completed.csv")

for (file.i in de.files) {
    file.i.qc.row <- c(file.i)
    
    ###----- Create output names & output directory
    file.i.outpath <- gsub(de.out.dir, "", file.i)
    file.i.outpath <- gsub(".csv", "", file.i.outpath)
    file.i.outpath <- gsub("^/", "",file.i.outpath)

    file.i.simp.name <- tail(unlist(strsplit( file.i.outpath, "/")), 1)
    file.i.simp.name <- gsub("_differential", "", file.i.simp.name)
    file.i.simp.vec <- unlist(strsplit(file.i.simp.name, "_vs_"))

    # Create output directory
    file.i.dir <- file.path(sum.dir, paste(file.i.outpath, "_sep", sep=""))
    dir.create(file.i.dir, recursive=TRUE, showWarnings=FALSE)
    
    for (gs.file.i in gs.files) {
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file.i))
        file.i.gs.dir <- file.path(file.i.dir, gs.file.base)
        dir.create(file.i.gs.dir, recursive=TRUE, showWarnings=FALSE)
        setwd(file.i.gs.dir)
        
        # Run GSEA analysis
        useGroup <- "t-test_overestim_var_logfc"
        outName <- file.i.simp.name
        
        success <- "yes"
        gsea <- tryCatch({
            GSEA_analysis(file.i, useGroup, outName, gs.file.i, 1)
        }, error=function(cond) {
            message(paste(file.i, gs.file.i,sep='; '))
            return("no")
        })
        
        if (!is.null(gsea)){
            success <- ""
        }
        
        file.i.qc.row <- c(file.i.qc.row, success)
    }
    
    completed.df[nrow(completed.df) +1, ] <- file.i.qc.row
    write.csv(completed.df, completed.name, row.names=TRUE)
}


── Column specification ───────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  P0 = col_double(),
  P1 = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“Unknown columns: `gene_name`”


[1] 3158



── Column specification ───────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (1.62% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ───────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  P0 = col_double(),
  P1 = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning message:
“U

[1] 3158



── Column specification ───────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (1.62% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO/1_Scanpy/0-all_Scanpy_out/2_DE/Cluster_vs_Cluster_newNames/P0/P0_vs_P1_differential.csv; /media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD4_mm_sigs.csv


── Column specification ───────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  P0 = col_double(),
  P1 = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-t

...


── Column specification ───────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (8.61% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ───────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  CD25KO = col_double(),
  CD25WT = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning mes

[1] 3468



── Column specification ───────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (8.61% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...


── Column specification ───────────────────────────────────────────────────
cols(
  gene_names = col_character(),
  CD25KO = col_double(),
  CD25WT = col_double(),
  wilcoxon_logfc = col_double(),
  wilcoxon_padj = col_double(),
  wilcoxon_score = col_double(),
  `t-test_logfc` = col_double(),
  `t-test_padj` = col_double(),
  `t-test_score` = col_double(),
  `t-test_overestim_var_logfc` = col_double(),
  `t-test_overestim_var_padj` = col_double(),
  `t-test_overestim_var_score` = col_double()
)


Warning mes

[1] 3468



── Column specification ───────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

GSEA analysis...

Warning message in fgsea(pathways = geneSets, stats = geneList, nperm = nPerm, minSize = minGSSize, :
“There are ties in the preranked stats (8.61% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
leading edge analysis...

done...

Warning message:
“Removed 2 rows containing missing values (geom_rect).”
Warning message:
“Removed 2 rows containing missing values (geom_rect).”
/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO/1_Scanpy/0-all_Scanpy_out/2_DE/perCluster_KO_vs_WT/P7/CD25WT/CD25WT_vs_CD25KO_differential.csv; /media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Wolski_curated_CD8_mm_sigs.csv

